Das CRISP DM Modell teilt sich in folgende Phasen auf:
<li> Business understanding
<li> Data understanding
<li> Data preparation
<li> Modeling
<li> Evaluating
<li> Deployment

In [63]:
# Benötigte Module importieren
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

<h2> 1. Business understanding </h2>

Wir betrachten das Titanic-Problem. Basierend auf Passagierdaten wollen wir vorhersagen, ob jemand den Untergang der Titanic überlebt hätte. 




<h2> 2. Data understanding </h2>
Der Datensatz enthält Merkmale wie Geschlecht, Alter, Passagierklasse, Ticketpreis, und Anzahl der Familienmitglieder an Bord. 

In [64]:
# Daten einlesen
df = pd.read_csv('titanic.csv', delimiter=",")

In [65]:
# Welche Spalten sind im Datensatz vorhanden? 
# Wie viele Zeilen sind im Datensatz vorhanden?
# Wie viele fehlende Werte sind im Datensatz vorhanden?
# Welche Datentypen sind im Datensatz vorhanden?
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [66]:
# Statistische Kennzahlen anzeigen
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


In [67]:
# Menge der fehlenden Werte in den Spalten
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

<h2>3. Data preparation </h2>
Die Daten werden zur Modellierung vorbereitet

In [68]:
# Der Datensatz wird aufgeteilt in die Zielvariable und die Features
df_target = df['Survived']
df = df.drop(columns = ["Name", "Survived", "PassengerId", "Cabin", "Ticket"], axis = 1)

In [69]:
# Wir ersetzen die fehlenden Werte in der Spalte "Age" durch den Medianwert der Spalte
median_value = df['Age'].median() 
df['Age'] = df['Age'].fillna(median_value)

df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_1984\1027524841.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)


In [70]:
# Die kategorischen Variablen werden in numerische Variablen umgewandelt
encoder = LabelEncoder()
categories = ['Sex', 'Embarked']

for i in categories:
    df[i] = encoder.fit_transform(df[i])

In [71]:
df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,2
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,2
3,1,0,35.0,1,0,53.1000,2
4,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...
886,2,1,27.0,0,0,13.0000,2
887,1,0,19.0,0,0,30.0000,2
888,3,0,28.0,1,2,23.4500,2
889,1,1,26.0,0,0,30.0000,0


<h2> 4. Modeling & Evaluierung </h2>
Es wird der K-nearest-neighbours Algorithmus angwendet, um die Daten zu modellieren. <br>
Für jeden Durchlauf des Modells wird eine Evaluierung durchgeführt.

In [72]:
states = [27, 6728, 49122]
features = df
target = df_target

In [75]:
accs = []

for RANDOM_STATE in states:
    # Daten aufteilen
    X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size=0.3, random_state=RANDOM_STATE
    )
    
    # Skalieren
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Modell erstellen
    knn = KNeighborsClassifier(n_neighbors=3)
    
    # Cross-Validation durchführen
    cv_scores = cross_val_score(knn, X_train, y_train, cv=10)
    mean_cv_score = np.mean(cv_scores)
    
    # Modell trainieren und testen
    knn.fit(X_train, y_train)
    preds = knn.predict(X_test)
    test_accuracy = accuracy_score(y_test, preds)
    accs.append(test_accuracy)
    
    # Ergebnisse ausgeben
    print(f"Random State: {RANDOM_STATE}")
    print(f"Cross-Validation Accuracy (Train): {mean_cv_score:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(classification_report(y_test, preds, zero_division=0.0))

# Durchschnittliche Genauigkeit und Standardabweichung
accs_mean = round(np.mean(accs), 4)
accs_std = round(np.std(accs), 4)

print(f"Mean Test Accuracy: {accs_mean}")
print(f"Std Test Accuracy: {accs_std}")


Random State: 27
Cross-Validation Accuracy (Train): 0.7976
Test Accuracy: 0.7985
              precision    recall  f1-score   support

           0       0.84      0.84      0.84       167
           1       0.73      0.73      0.73       101

    accuracy                           0.80       268
   macro avg       0.79      0.79      0.79       268
weighted avg       0.80      0.80      0.80       268

Random State: 6728
Cross-Validation Accuracy (Train): 0.7847
Test Accuracy: 0.8134
              precision    recall  f1-score   support

           0       0.87      0.82      0.85       168
           1       0.73      0.80      0.76       100

    accuracy                           0.81       268
   macro avg       0.80      0.81      0.80       268
weighted avg       0.82      0.81      0.82       268

Random State: 49122
Cross-Validation Accuracy (Train): 0.7800
Test Accuracy: 0.7649
              precision    recall  f1-score   support

           0       0.82      0.80      0.81

<h2> 5. Deployment </h2>

<li>Zunächst wurden die Daten aufbereitet, um ein Verständnis für sie zu entwickeln. Es lagen Daten im numerischen und kategorischen Format vor
<li>Wir haben verschiedene Datensätze ausprobiert und getestet, um die höchste Genauigkeit zu erzielen, und sind zu dem Schluss gekommen, dass die Merkmale Pclass, Sex, Age, SibSP, Parch, Ticket, Fare und Embarked die besten Ergebnisse liefern
<li>Bspw. wurde das Feature "Cabin" entfernt, da hier ein großteil der Daten gefehlt hat
<li>Die Ergebnisse der KNN-Modelle zur Vorhersage des Überlebens der Titanic-Katastrophe zeigen eine durchschnittliche Testgenauigkeit von 82,59% mit geringer Schwankung (±1,5%). Die beste Genauigkeit wurde bei Random State 27 mit 84,70% erreicht.</li>
<li>Die Cross-Validation-Ergebnisse unterstützen diese Beobachtungen, wobei das Modell für Random State 27 eine durchschnittliche Genauigkeit von 79,92% über die Validierungsfolds erreichte. Dies zeigt, dass das Modell auch in anderen Datenaufteilungen robust ist
<li>Das Modell zeigt eine starke Leistung bei der Erkennung von Nicht-Überlebenden (Klasse 0) mit einem Recall von 92% und einer Präzision von 85%. Die Vorhersage von Überlebenden (Klasse 1) ist schwächer, mit einem Recall von 72% bei ebenfalls 85% Präzision.</li>
<li>Insgesamt liefert das KNN-Modell robuste Vorhersagen, besonders für Nicht-Überlebende, mit Potenzial zur Optimierung bei der Erkennung von Überlebenden.</li>